<H2> INTRODUCTION

 - This script reads in the required files from AWS (for the challenge) and exports it to the local environment for usage.
 - Link to CPT DS challenge: https://github.com/cityofcapetown/ds_code_challenge
 - Submission for the CCT Data Scientist role, July 2022.
 - Compiler/submitter: Ryan Williams

<H3> IMPORT DEPENDANCIES

In [1]:
import json
import botocore
import boto3
import io
from io import BytesIO
import pandas as pd
import gzip
import geopandas as gpd
from area import area
import shapely
from shapely.geometry import Polygon
import datetime

<h3> SETUP CREDENTIALS AND AWS SESSION

In [2]:
# Load secrets file:
with open('secrets.json') as f:
    json_data = json.load(f)
aws_access_key_id=json_data['s3']['access_key']
aws_secret_access_key=json_data['s3']['secret_key']
region='af-south-1'

# aws_access_key_id,aws_secret_access_key,region #Debug.

In [3]:
s3_client = boto3.client(
    "s3",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region
)

In [4]:
bucket_name = 'cct-ds-code-challenge-input-data'

---

<h3> READ IN FILES AND EXPORT

**Read the "sr.csv.gz" file**

In [16]:
response = s3_client.get_object(Bucket=bucket_name, Key='sr.csv.gz')
gzipfile = gzip.GzipFile(fileobj=BytesIO(response['Body'].read()))
sr=pd.read_csv(BytesIO(gzipfile.read()))
sr

,Unnamed: 0,notification_number,reference_number,creation_timestamp,completion_timestamp,directorate,department,branch,section,code_group,code,cause_code_group,cause_code,official_suburb,latitude,longitude
0,0,400583534,9.109492e+09,2020-10-07 06:55:18+02:00,2020-10-08 15:36:35+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area Central,District: Blaauwberg,TD Customer complaint groups,Pothole&Defect Road Foot Bic Way/Kerbs,Road (RCL),Wear and tear,MONTAGUE GARDENS,-33.872839,18.522488
1,1,400555043,9.108995e+09,2020-07-09 16:08:13+02:00,2020-07-14 14:27:01+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area East,District : Somerset West,TD Customer complaint groups,Manhole Cover/Gully Grid,Road (RCL),Vandalism,SOMERSET WEST,-34.078916,18.848940
2,2,400589145,9.109614e+09,2020-10-27 10:21:59+02:00,2020-10-28 17:48:15+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area East,District : Somerset West,TD Customer complaint groups,Manhole Cover/Gully Grid,Road (RCL),Vandalism,STRAND,-34.102242,18.821116
3,3,400538915,9.108601e+09,2020-03-19 06:36:06+02:00,2021-03-29 20:34:19+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area North,District : Bellville,TD Customer complaint groups,Paint Markings Lines&Signs,Road Markings,Wear and tear,RAVENSMEAD,-33.920019,18.607209
4,4,400568554,NaN,2020-08-25 09:48:42+02:00,2020-08-31 08:41:13+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area South,District : Athlone,TD Customer complaint groups,Pothole&Defect Road Foot Bic Way/Kerbs,Road (RCL),Surfacing failure,CLAREMONT,-33.987400,18.453760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941629,941629,1016508425,9.109974e+09,2020-12-31 23:49:38+02:00,2021-01-11 11:54:42+02:00,WATER AND SANITATION,Distribution Services,Reticulation,Reticulation Water Distribution,WATER,Leak at Valve,NaN,NaN,WOODSTOCK,-33.931571,18.452159
941630,941630,1016508432,9.109975e+09,2020-12-31 23:31:11+02:00,2021-01-04 11:46:28+02:00,WATER AND SANITATION,Distribution Services,Reticulation,NaN,SEWER,Sewer: Blocked/Overflow,General,Foreign Objects,FISANTEKRAAL,-33.783246,18.716554
941631,941631,1016508434,9.109975e+09,2020-12-31 23:58:21+02:00,2021-01-01 00:01:08+02:00,WATER AND SANITATION,Distribution Services,Reticulation,NaN,WATER,Burst Pipe,NaN,NaN,NaN,NaN,NaN
941632,941632,1016508442,9.109975e+09,2020-12-31 23:41:57+02:00,2021-01-05 15:59:24+02:00,WATER AND SANITATION,Commercial Services,Customer Services (Water),Meter Management,WATER MANAGEMENT DEVICE,No Water WMD,NaN,NaN,WESBANK,-33.971099,18.659831


In [17]:
# Specify date and name and export the file.
date_time=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
name=date_time+'_service_requests.csv'

sr.to_csv(name)

**Read the "sr_hex.csv.gz" file**

In [18]:
response = s3_client.get_object(Bucket=bucket_name, Key='sr_hex.csv.gz')
gzipfile = gzip.GzipFile(fileobj=BytesIO(response['Body'].read()))
sr_hex=pd.read_csv(BytesIO(gzipfile.read()))
sr_hex

,notification_number,reference_number,creation_timestamp,completion_timestamp,directorate,department,branch,section,code_group,code,cause_code_group,cause_code,official_suburb,latitude,longitude,h3_level8_index
0,400583534,9.109492e+09,2020-10-07 06:55:18+02:00,2020-10-08 15:36:35+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area Central,District: Blaauwberg,TD Customer complaint groups,Pothole&Defect Road Foot Bic Way/Kerbs,Road (RCL),Wear and tear,MONTAGUE GARDENS,-33.872839,18.522488,88ad360225fffff
1,400555043,9.108995e+09,2020-07-09 16:08:13+02:00,2020-07-14 14:27:01+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area East,District : Somerset West,TD Customer complaint groups,Manhole Cover/Gully Grid,Road (RCL),Vandalism,SOMERSET WEST,-34.078916,18.848940,88ad36d5e1fffff
2,400589145,9.109614e+09,2020-10-27 10:21:59+02:00,2020-10-28 17:48:15+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area East,District : Somerset West,TD Customer complaint groups,Manhole Cover/Gully Grid,Road (RCL),Vandalism,STRAND,-34.102242,18.821116,88ad36d437fffff
3,400538915,9.108601e+09,2020-03-19 06:36:06+02:00,2021-03-29 20:34:19+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area North,District : Bellville,TD Customer complaint groups,Paint Markings Lines&Signs,Road Markings,Wear and tear,RAVENSMEAD,-33.920019,18.607209,88ad361133fffff
4,400568554,NaN,2020-08-25 09:48:42+02:00,2020-08-31 08:41:13+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area South,District : Athlone,TD Customer complaint groups,Pothole&Defect Road Foot Bic Way/Kerbs,Road (RCL),Surfacing failure,CLAREMONT,-33.987400,18.453760,88ad361709fffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941629,1016508425,9.109974e+09,2020-12-31 23:49:38+02:00,2021-01-11 11:54:42+02:00,WATER AND SANITATION,Distribution Services,Reticulation,Reticulation Water Distribution,WATER,Leak at Valve,NaN,NaN,WOODSTOCK,-33.931571,18.452159,88ad361547fffff
941630,1016508432,9.109975e+09,2020-12-31 23:31:11+02:00,2021-01-04 11:46:28+02:00,WATER AND SANITATION,Distribution Services,Reticulation,NaN,SEWER,Sewer: Blocked/Overflow,General,Foreign Objects,FISANTEKRAAL,-33.783246,18.716554,88ad3656d7fffff
941631,1016508434,9.109975e+09,2020-12-31 23:58:21+02:00,2021-01-01 00:01:08+02:00,WATER AND SANITATION,Distribution Services,Reticulation,NaN,WATER,Burst Pipe,NaN,NaN,NaN,NaN,NaN,0
941632,1016508442,9.109975e+09,2020-12-31 23:41:57+02:00,2021-01-05 15:59:24+02:00,WATER AND SANITATION,Commercial Services,Customer Services (Water),Meter Management,WATER MANAGEMENT DEVICE,No Water WMD,NaN,NaN,WESBANK,-33.971099,18.659831,88ad36c49bfffff


In [19]:
# Specify date and name and export the file.
date_time=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
name=date_time+'_service_requests_8hex.csv'

sr_hex.to_csv(name)

**Read the "sr_hex_truncated.csv" file**

In [7]:
response = s3_client.get_object(Bucket=bucket_name, Key='sr_hex_truncated.csv')
sr_hex_trunk = pd.read_csv(response.get("Body"))
sr_hex_trunk

,notification_number,reference_number,creation_timestamp,completion_timestamp,directorate,department,branch,section,code_group,code,cause_code_group,cause_code,official_suburb,latitude,longitude,h3_level8_index
0,400538915,9.108601e+09,2020-03-19 06:36:06+02:00,2021-03-29 20:34:19+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area North,District : Bellville,TD Customer complaint groups,Paint Markings Lines&Signs,Road Markings,Wear and tear,RAVENSMEAD,-33.920019,18.607209,88ad361133fffff
1,400531832,9.108432e+09,2020-02-18 07:28:11+02:00,2020-02-25 07:48:44+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area South,District : Athlone,TD Customer complaint groups,Paint Markings Lines&Signs,Road (RCL),Wear and tear,BRIDGETOWN,-33.959023,18.535687,88ad3610e9fffff
2,400538969,9.108603e+09,2020-03-19 08:19:43+02:00,2021-03-09 20:59:23+02:00,NaN,NaN,NaN,NaN,TD Customer complaint groups,Pothole&Defect Road Foot Bic Way/Kerbs,Road (RCL),Base failure,RICHWOOD,-33.835900,18.541292,88ad360337fffff
3,400525122,NaN,2020-01-22 14:18:42+02:00,NaN,WATER AND SANITATION,Distribution Services,Reticulation,Reticulation WW Conveyance,TD Customer complaint groups,Pothole&Defect Road Foot Bic Way/Kerbs,Sidewalk,Underground service failure,GRASSY PARK,-34.056338,18.494952,88ad368d11fffff
4,400539785,NaN,2020-03-24 09:00:38+02:00,2022-02-21 07:27:11+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area North,District : Bellville,TD Customer complaint groups,Manhole Cover/Gully Grid,NaN,NaN,FLORIDA,-33.922589,18.594716,88ad3611e1fffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254799,1015749632,9.108652e+09,2020-03-31 22:50:30+02:00,2020-04-03 08:21:29+02:00,ENERGY,Electricity Generation and Distribution,Enterprise Asset Management,CTE Distribution East,ELECTRICITY TECHNICAL COMPLAINTS,No Power,Point of Supply,Faulty,RAVENSMEAD,-33.926694,18.603812,88ad3611edfffff
254800,1015749633,9.108652e+09,2020-03-31 23:10:38+02:00,2020-04-02 08:54:34+02:00,URBAN WASTE MANAGEMENT,Solid Waste Management,Cleansing,NaN,SOLID WASTE,Illegal Dumping,NaN,NaN,RETREAT,-34.057951,18.485519,88ad368d1bfffff
254801,1015749643,NaN,2020-03-31 23:12:43+02:00,2020-09-12 11:55:39+02:00,ENERGY,Electricity Generation and Distribution,Enterprise Asset Management,CTE Distribution South,ELECTRICITY TECHNICAL COMPLAINTS,No Power,NaN,NaN,CLAREMONT,-33.981874,18.480294,88ad361761fffff
254802,1015749644,NaN,2020-03-31 23:12:12+02:00,2020-04-03 08:18:15+02:00,ENERGY,Electricity Generation and Distribution,Enterprise Asset Management,CTE Distribution South,ELECTRICITY TECHNICAL COMPLAINTS,PPM Faulty Mode,NaN,NaN,ROCKLANDS,-34.062772,18.602571,88ad3688a5fffff


In [8]:
# Specify date and name and export the file.
date_time=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
name=date_time+'_service_requests_8hex_truncated.csv'

sr_hex_trunk.to_csv(name)

**Read the "city-hex-polygons-8.geojson" file**

In [22]:
response = s3_client.get_object(Bucket=bucket_name, Key='city-hex-polygons-8.geojson')
hex8 = gpd.read_file(response.get("Body"))
hex8

,index,centroid_lat,centroid_lon,geometry
0,88ad361801fffff,-33.859427,18.677843,"POLYGON ((18.68119 -33.86330, 18.68357 -33.859..."
1,88ad361803fffff,-33.855696,18.668766,"POLYGON ((18.67211 -33.85957, 18.67450 -33.855..."
2,88ad361805fffff,-33.855263,18.685959,"POLYGON ((18.68931 -33.85914, 18.69169 -33.855..."
3,88ad361807fffff,-33.851532,18.676881,"POLYGON ((18.68023 -33.85541, 18.68261 -33.851..."
4,88ad361809fffff,-33.867322,18.678806,"POLYGON ((18.68215 -33.87120, 18.68454 -33.867..."
...,...,...,...,...
3827,88ad369715fffff,-34.353404,18.479198,"POLYGON ((18.48255 -34.35726, 18.48494 -34.353..."
3828,88ad369717fffff,-34.349672,18.470112,"POLYGON ((18.47346 -34.35353, 18.47585 -34.349..."
3829,88ad369733fffff,-34.337717,18.477288,"POLYGON ((18.48063 -34.34158, 18.48303 -34.337..."
3830,88ad369739fffff,-34.349293,18.487330,"POLYGON ((18.49068 -34.35315, 18.49307 -34.349..."


In [23]:
hex8.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [24]:
# Specify date and name and export the file.
date_time=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
name=date_time+'_cpt8hex_geometries.geojson'

hex8.to_file(name, driver='GeoJSON')

C:\Users\ryanp\anaconda3\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


**Read the "city-hex-polygons-8-10.geojson" file**

In [25]:
response = s3_client.get_object(Bucket=bucket_name, Key='city-hex-polygons-8-10.geojson')
hex8_10 = gpd.read_file(response.get("Body"))
hex8_10

,index,centroid_lat,centroid_lon,resolution,geometry
0,88ad361801fffff,-33.859427,18.677843,8,"POLYGON ((18.68119 -33.86330, 18.68357 -33.859..."
1,88ad361803fffff,-33.855696,18.668766,8,"POLYGON ((18.67211 -33.85957, 18.67450 -33.855..."
2,88ad361805fffff,-33.855263,18.685959,8,"POLYGON ((18.68931 -33.85914, 18.69169 -33.855..."
3,88ad361807fffff,-33.851532,18.676881,8,"POLYGON ((18.68023 -33.85541, 18.68261 -33.851..."
4,88ad361809fffff,-33.867322,18.678806,8,"POLYGON ((18.68215 -33.87120, 18.68454 -33.867..."
...,...,...,...,...,...
203835,8aad36973b6ffff,-34.348444,18.474894,10,"POLYGON ((18.47537 -34.34899, 18.47571 -34.348..."
203836,8aad36973b77fff,-34.348389,18.477354,10,"POLYGON ((18.47783 -34.34894, 18.47817 -34.348..."
203837,8aad36973b8ffff,-34.345507,18.480703,10,"POLYGON ((18.48118 -34.34606, 18.48152 -34.345..."
203838,8aad36973ba7fff,-34.347161,18.482137,10,"POLYGON ((18.48262 -34.34771, 18.48296 -34.347..."


In [26]:
hex8_10.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [27]:
# Specify date and name and export the file.
date_time=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
name=date_time+'_cpt8_10hex_geometries.geojson'

hex8_10.to_file(name, driver='GeoJSON')

C:\Users\ryanp\anaconda3\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
